In [2]:
#importing libraries
import numpy as np 
import torch 
import torch.nn as nn
import torch.nn.functional as F 

### Gating Mechanism

In [4]:
class Gate(nn.Module):
    def __init__(self, input_dim, num_experts, k=1):
        super().__init__()
        self.k = k
        self.gate = nn.Linear(input_dim, num_experts)

    def forward(self, X):
        logits = self.gate(X)
        k_vals, k_indices = torch.topk(logits, self.k, dim = -1)
        scores = F.softmax(k_vals, dim = -1)
        return k_indices, scores

### Experts

In [ ]:
class Experts(nn.Module):
    def __init__(self, num_experts, input_dim, hidden_dim):
        super().__init__()
        self.experts = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, input_dim)
            ) for _ in range(num_experts)
        ])

    def forward(self, X, indices):
        batch_size, k = indices.shape
        out = torch.zeros(batch_size, X.shapep[1], device = X.device)
        for i in range(k):
            expert_idx = indices[:,i]
            